In [5]:
import iotadas as adas
import cv2
import os
import imutils
from tkinter import *
import os
import cv2
from matplotlib import pyplot
import numpy as np
from time import sleep
import threading
from tkinter import  font,ttk
import tkinter as  tk
 
import multiprocessing
def check_file_existance(filePath):
    try:
        with open(filePath, 'r') as f:
            return True
    except FileNotFoundError as e:
        return False
    except IOError as e:
        return False
    
def user_image_capturer (user,dataPath):
    if user is not None:
        personName = user
        personPath = dataPath + '/' + personName

        if not os.path.exists(personPath):
            print('Carpeta creada: ',personPath)
            os.makedirs(personPath)

        cap = cv2.VideoCapture(0,cv2.CAP_DSHOW)
        
        faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
        count = 0

        while True:

            ret, frame = cap.read()
            if ret == False: break
            frame =  imutils.resize(frame, width=640)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            auxFrame = frame.copy()

            faces = faceClassif.detectMultiScale(gray,1.3,5)

            for (x,y,w,h) in faces:
                cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
                rostro = auxFrame[y:y+h,x:x+w]
                rostro = cv2.resize(rostro,(150,150),interpolation=cv2.INTER_CUBIC)
                cv2.imwrite(personPath + '/rotro_{}.jpg'.format(count),rostro)
                count = count + 1
            cv2.imshow('frame',frame)

            k =  cv2.waitKey(1)
            if k == 27 or count >= 500:
                break

        cap.release()
        cv2.destroyAllWindows()
        return True
    else: 
        return False


def train_user (user,dataPath,modelPath):
    
    peopleList = dataPath+'/'+user #os.listdir(dataPath)
    print('Lista de personas: ', peopleList)
    
    labels = []
    facesData = []
    label = 0

    #for nameDir in peopleList:
    
    personPath = dataPath + '/' + user
    print('Leyendo las imágenes')

    for fileName in os.listdir (personPath):
        print('Rostros: ', personPath + '/' + fileName)
        labels.append(label)
        facesData.append(cv2.imread(personPath+'/'+fileName,0))
        #image = cv2.imread(personPath+'/'+fileName,0)
        #cv2.imshow('image',image)
        #cv2.waitKey(10)
    label = label + 1

    # Métodos para entrenar el reconocedor
    face_recognizer = cv2.face.LBPHFaceRecognizer_create()

    # Entrenando el reconocedor de rostros
    print("Entrenando...")
    face_recognizer.train(facesData, np.array(labels))

    # Almacenando el modelo obtenido
    model = modelPath+user+'_'+'modeloLBPHFace.xml'
    print(" model :",model)
    

    if not os.path.exists(modelPath):
        print('Carpeta creada: ',modelPath)
        os.makedirs(modelPath)
            
    face_recognizer.write(model)
    print(" Usario Registrado...")

def user_detect (user,dataPath,modelPath):
    
    detect = False
    if user is not None:
        imagePaths = os.listdir(dataPath)
        print('imagePaths=',imagePaths)

        #face_recognizer = cv2.face.EigenFaceRecognizer_create()
        #face_recognizer = cv2.face.FisherFaceRecognizer_create()
        face_recognizer = cv2.face.LBPHFaceRecognizer_create()

        # Leyendo el modelo
        #face_recognizer.read('modeloEigenFace.xml')
        #face_recognizer.read('modeloFisherFace.xml')
        model = modelPath+user+'_'+'modeloLBPHFace.xml'
        if check_file_existance (model):
            print (' [*] model:',model)
            face_recognizer.read(model)

            cap = cv2.VideoCapture(0,cv2.CAP_DSHOW)
            #cap = cv2.VideoCapture('Video.mp4')

            faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')

            while True:
                ret,frame = cap.read()
                if ret == False: break
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                auxFrame = gray.copy()

                faces = faceClassif.detectMultiScale(gray,1.3,5)

                for (x,y,w,h) in faces:
                    rostro = auxFrame[y:y+h,x:x+w]
                    rostro = cv2.resize(rostro,(150,150),interpolation= cv2.INTER_CUBIC)
                    result = face_recognizer.predict(rostro)
                    print( ' Predict user:',result)


                    cv2.putText(frame,'{}'.format(result),(x,y-5),1,1.3,(255,0,255),1,cv2.LINE_AA)

                    '''
                    # EigenFaces
                    if result[1] < 5700:
                        cv2.putText(frame,'{}'.format(imagePaths[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv2.LINE_AA)
                        cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
                    else:
                        cv2.putText(frame,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv2.LINE_AA)
                        cv2.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)

                    # FisherFace
                    if result[1] < 500:
                        cv2.putText(frame,'{}'.format(imagePaths[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv2.LINE_AA)
                        cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
                    else:
                        cv2.putText(frame,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv2.LINE_AA)
                        cv2.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)
                    '''
                    # LBPHFace
                    if result[1] < 70:
                        cv2.putText(frame,'{}'.format(imagePaths[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv2.LINE_AA)
                        cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
                        detect = True
                    else:
                        cv2.putText(frame,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv2.LINE_AA)
                        cv2.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)
                        detect = False

                cv2.imshow('frame',frame)
                k = cv2.waitKey(1)
                if k == 27:
                    break
                    
            cap.release()
            cv2.destroyAllWindows()      
            return detect
        
        else:
            return detect
    else: 
        return detect  

def reg_user (user):
    dataPath    =  './data/reconocimiento' #directorio almacenamiento fotos usuarios para entrenamiento
    modelPath   =  './model/user/'
    if (user != '') and user_image_capturer (user,dataPath):
        train_user (user,dataPath,modelPath)
        return True
    else:
        return False
    
def login_user (user):
    print('login_user:[',user,']')
    dataPath    =  './data/reconocimiento' #directorio almacenamiento fotos usuarios para entrenamiento
    modelPath   =  './model/user/'
    if ((user != '') and (user is not None)):
        if user_detect (user,dataPath,modelPath):
             return True
        else:
            return False
    else:
        return False
# ----------------------------------------------
#  ---------------- Front-End APPP  ------------
# -----------------------------------------------

def registrar_usuario():
    path_users = "./data/users/"
    usuario_info = usuario.get()         # Obetnemos la informacion alamcenada en usuario
    contra_info  = contra.get()          # Obtenemos la informacion almacenada en contra
    
    if not os.path.exists(path_users):
        print('Carpeta creada: ',path_users)
        os.makedirs(path_users)
    file_user = path_users + usuario_info

    archivo = open(file_user, "w")    # Abriremos la informacion en modo escritura
    archivo.write(usuario_info + "\n")   # escribimos la info
    archivo.write(contra_info)
    archivo.close()

    #Limpiaremos los text variable
    usuario_entrada.delete(0, END)
    contra_entrada.delete(0, END)

    #Ahora le diremos al usuario que su registro ha sido exitoso
    Label(pantalla1, text = "Registro Convencional Exitoso", fg = "green", font = ("Calibri",11)).pack()
    sleep(3)
    pantalla1.destroy()
    pantalla.state('normal') 

#--------------------------- Funcion para almacenar el registro facial --------------------------------------
def registro_facial():
    usuario_img = usuario.get()
    if usuario_img != '' and reg_user (usuario_img):
    
        usuario_entrada.delete(0, END)   #Limpiamos los text variables
        contra_entrada.delete(0, END)
        Label(pantalla1, text = "Registro Facial Exitoso", fg = "green", font = ("Calibri",11)).pack()
        sleep (2)
        # Inicio Sesión mediante bluetooth
        pantalla1.destroy()
        pantalla.state('normal') 
  
    else:
        Label(pantalla1, text = "Error Registro!!", fg = "green", font = ("Calibri",11)).pack()
        Label(pantalla1, text = "compruebe si ha intrducido el usuario", fg = "green", font = ("Calibri",11)).pack()
 #------------------------Crearemos una funcion para asignar al boton registro --------------------------------
def registro():
    global usuario
    global contra  #Globalizamos las variables para usarlas en otras funciones
    global usuario_entrada
    global contra_entrada
    global pantalla1
    
    pantalla1 = Toplevel(pantalla) #Esta pantalla es de un nivel superior a la principal
    pantalla.iconify()
    pantalla1.title("Registro")
    pantalla1.geometry("500x450")   #Creamos la ventana
    #--------- Empezaremos a crear las entradas ----------------------------------------
    
    usuario = StringVar()
    contra = StringVar()
    
    Label(pantalla1, text = "Registro facial: debe de asignar un usuario:").pack()
    #Label(pantalla1, text = "").pack()  #Dejamos un poco de espacio
    Label(pantalla1, text = "Registro tradicional: debe asignar usuario y contraseña:").pack()
    Label(pantalla1, text = "").pack()  #Dejamos un poco de espacio
    Label(pantalla1, text = "Usuario * ").pack()  #Mostramos en la pantalla 1 el usuario
    usuario_entrada = Entry(pantalla1, textvariable = usuario) #Creamos un text variable para que el usuario ingrese la info
    usuario_entrada.pack()
    Label(pantalla1, text = "Contraseña * ").pack()  #Mostramos en la pantalla 1 la contraseña
    contra_entrada = Entry(pantalla1, textvariable = contra) #Creamos un text variable para que el usuario ingrese la contra
    contra_entrada.pack()
    Label(pantalla1, text = "").pack()  #Dejamos un espacio para la creacion del boton
    Button(pantalla1, text = "Registro Tradicional", width = 15, height = 1, command = registrar_usuario).pack()  #Creamos el boton

    #------------ Vamos a crear el boton para hacer el registro facial --------------------
    Label(pantalla1, text = "").pack()
    Button(pantalla1, text = "Registro Facial", width = 15, height = 1, command = registro_facial).pack()
    # create button to implement destroy()
    Label(pantalla1,text = "").pack()  #Creamos el espacio  
    Button(pantalla1, text="Quit", command=pantalla1.destroy).pack()
#------------------------------------------- Funcion para verificar los datos ingresados al login ------------------------------------
    
def verificacion_login():
    path_users = "./data/users"
    log_usuario = verificacion_usuario.get()
    log_contra = verificacion_contra.get()

    usuario_entrada2.delete(0, END)
    contra_entrada2.delete(0, END)

    lista_archivos = os.listdir(path_users)   #Vamos a importar la lista de archivos con la libreria os
    print(lista_archivos)
    if log_usuario in lista_archivos:   #Comparamos los archivos con el que nos interesa
        try:
            pt_user  = path_users + "/" + log_usuario
            archivo2 = open(pt_user, "r")  #Abrimos el archivo en modo lectura
            verificacion = archivo2.read().splitlines()  #leera las lineas dentro del archivo ignorando el resto
            if log_contra in verificacion:
                print("Inicio de sesion exitoso")
                Label(pantalla2, text = "Inicio de Sesion Exitoso", fg = "green", font = ("Calibri",11)).pack()
                # ------------------- Inicio de la app -----------------
                usuario_entrada2.delete(0, END)   #Limpiamos los text variables
                contra_entrada2.delete(0, END)
                sleep(5)
                pantalla2.destroy()
                pantalla.state('normal') 

                init_app ()            
            else:
                print("Contraseña incorrecta, ingrese de nuevo")
                Label(pantalla2, text = "Contraseña Incorrecta", fg = "red", font = ("Calibri",11)).pack()
        except:
            print("Directorio de Usuario no encontrado")
            Label(pantalla2, text = "Directorio de Usuario no encontrado", fg = "red", font = ("Calibri",11)).pack()
    else:
        print("Usuario no encontrado")
        Label(pantalla2, text = "Usuario no encontrado", fg = "red", font = ("Calibri",11)).pack()
    
#--------------------------Funcion para el Login Facial --------------------------------------------------------
def login_facial ():
    usuario_login = verificacion_usuario.get()    #Con esta variable vamos a guardar la foto pero con otro nombre para no sobreescribir
    print('login_facial ():',usuario_login)
    if login_user (usuario_login):
        print("Inicio de sesion exitoso")
        Label(pantalla2, text = "Inicio de Sesion Exitoso", fg = "green", font = ("Calibri",11)).pack()
         # ------------------- Inicio de la app -----------------
        usuario_entrada2.delete(0, END)   #Limpiamos los text variables
        contra_entrada2.delete(0, END)
        sleep(5)
        pantalla2.destroy()
        pantalla.state('normal') 
 
        init_app ()
    else:
        print("Usuario no encontrado")
        Label(pantalla2, text = "Usuario no encontrado", fg = "red", font = ("Calibri",11)).pack()
        Label(pantalla2, text = "Compruebe e usuario introducido", fg = "red", font = ("Calibri",11)).pack()

#------------------------Funcion que asignaremos al boton login -------------------------------------------------
        
def login ():
    global pantalla2
    global verificacion_usuario
    global verificacion_contra
    global usuario_entrada2
    global contra_entrada2
    
    pantalla2 = Toplevel(pantalla)
    pantalla.iconify()
    pantalla2.title("Login")
    pantalla2.geometry("500x450")   #Creamos la ventana
    Label(pantalla2, text = "Login facial: debe de asignar un usuario:").pack()
    Label(pantalla2, text = "Login tradicional: debe asignar usuario y contraseña:").pack()
    Label(pantalla2, text = "").pack()  #Dejamos un poco de espacio
    
    verificacion_usuario = StringVar()
    verificacion_contra = StringVar()
    
    #---------------------------------- Ingresamos los datos --------------------------
    Label(pantalla2, text = "Usuario * ").pack()
    usuario_entrada2 = Entry(pantalla2, textvariable = verificacion_usuario)
    usuario_entrada2.pack()
    Label(pantalla2, text = "Contraseña * ").pack()
    contra_entrada2 = Entry(pantalla2, textvariable = verificacion_contra)
    contra_entrada2.pack()
    Label(pantalla2, text = "").pack()
    Button(pantalla2, text = "Inicio de Sesion Tradicional", width = 20, height = 1, command = verificacion_login).pack()

    #------------ Vamos a crear el boton para hacer el login facial --------------------
    Label(pantalla2, text = "").pack()
    Button(pantalla2, text = "Inicio de Sesion Facial", width = 20, height = 1, command = login_facial).pack()
    # create button to implement destroy()
    Label(pantalla2,text = "").pack()  #Creamos el espacio  
    Button(pantalla2, text="Quit", command=pantalla2.destroy).pack()  
    
  
#------------------------- Funcion de nuestra pantalla principal ------------------------------------------------

def pantalla_principal ():
    global pantalla          #Globalizamos la variable para usarla en otras funciones
    global butt_login
    global but_reg
    global pantalla_debug
    global debug_str
    
    
    pantalla = Tk()
    debug_str =  tk.StringVar() 
    
   
    pantalla.geometry("600x450")  #Asignamos el tamaño de la ventana 
    pantalla.title("Proyecto IOT GCD -UV 2022/23")       #Asignamos el titulo de la pantalla
    Label(text = "APP Login", bg = "gray", width = "300", height = "2", font = ("Verdana", 13)).pack() #Asignamos caracteristicas de la ventana
    
    
#------------------------- Vamos a Crear los Botones  si tenemos conexión comunicación UART para Bluetooth------------------------------------------------------
 
    Label(text = "").pack()  #Creamos el espacio entre el titulo y el primer boton
    butt_login = Button(text = "Iniciar Sesion", height = "2", width = "30", command = login)
    butt_login.pack()
    Label(text = "").pack() #Creamos el espacio entre el primer boton y el segundo boton
    but_reg = Button(text = "Registro", height = "2", width = "30", command = registro) 
    but_reg.pack()
    # create button to implement destroy()
    Label(text = "").pack()  #Creamos el espacio  
    Button(text="Exit", command=exit_app).pack()
    

 
    pantalla.mainloop()

def exit_app ():
    print("exit_app ()")
     
    #Enviamos para que se desconecte d HiveMqtt y topic
    adas.off_subs_app_mqttonly ()
    print(adas.isConnect())
    if adas.isConnect():
        adas.disconnect_mqtt ()
        adas.disconnect_client_cam()
        
    if 'sub_server' in globals():
        sub_server.join ()
    if 'r_somno' in globals():
        event.set()
        r_somno.join ()
  
    pantalla.destroy ()
#------------------------- Función de inicio APP : Enviamos por ------------------------------------------------------
def ini_adas ():
    adas.init_subs_app_mqttonly ()
def init_app ():
    global sub_server
    global r_somno
    global sub_server
    global event
    #Limpiamos los botones
    print("init_app ()")
    butt_login.pack_forget()
    but_reg.pack_forget()
    
    # Crear caja de texto.
    #pantalla_debug =  ttk.Entry(  state="readonly",takefocus=False,textvariable=debug_str,font=font.Font(family="Verdana", size=10))
    # Posicionarla en la ventana.
    #pantalla_debug.place(  x= 50, y= 50,width=500, height=400 )
    event = multiprocessing.Event() 
    r_somno = threading.Thread(target = adas.run_drowsiness,args=(event,))
    r_somno.start ()
    #pantalla_debug.pack()
    sleep(1)
    sub_server = threading.Thread(target = ini_adas,args=(  ))
    sub_server.start ()
    
    print("Estoy esperando en el topic para procesar los mensajes entrantes")
 
#----------------------------------------------------
pantalla_principal ()

['qq']
Inicio de sesion exitoso
init_app ()
Voy a conectar el cliente publicador de eventos de la cam
dbd25372b1b647e895c3f4adb0260f4e.s2.eu.hivemq.cloud
8883
CLient_Cam::>>> CONNACK received with code Success.
CLient_Cam::>>> Connect dbd25372b1b647e895c3f4adb0260f4e.s2.eu.hivemq.cloud result is: Success
CLient_Cam::>>> Connected OK
Estoy esperando en el topic para procesar los mensajes entrantes
NO CONECTA
[INFO] loading facial landmark predictor...


2023-01-07 00:16:01,887 root            INFO     send_cloudmseg


Servidor::>>> CONNACK received with code Success.
Servidor::>>> Connect dbd25372b1b647e895c3f4adb0260f4e.s2.eu.hivemq.cloud result is: Success
Servidor::>>> Connected OK
exit_app ()
Enevio mensaje LOGOUT NOK


2023-01-07 00:16:01,888 root            INFO     ['{"clientid": "Client_01", "action": "DTC", "event": "LOGIN", "status": "NOK"}']
2023-01-07 00:16:01,888 root            INFO     {"clientid": "Client_01", "action": "DTC", "event": "LOGIN", "status": "NOK"}


Enevio mensaje LOGIN OK


2023-01-07 00:16:03,353 root            CRITICAL event:INFO,Blinks:0,COUNTER:0,EAR:0.279758850839081
2023-01-07 00:16:03,356 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,mar:0.0
2023-01-07 00:16:03,406 root            CRITICAL event:INFO,Blinks:0,COUNTER:0,EAR:0.30830483285740684
2023-01-07 00:16:03,407 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,mar:0.024634832269113213
2023-01-07 00:16:03,445 root            CRITICAL event:INFO,Blinks:0,COUNTER:0,EAR:0.2948793290846106
2023-01-07 00:16:03,446 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,mar:0.0
2023-01-07 00:16:03,490 root            CRITICAL event:INFO,Blinks:0,COUNTER:0,EAR:0.32373776673390986
2023-01-07 00:16:03,491 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,mar:0.010201957331259656
2023-01-07 00:16:03,535 root            CRITICAL event:INFO,Blinks:0,COUNTER:0,EAR:0.31606267645106184
2023-01-07 00:16:03,536 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,mar:0.01041666666

Mensaje LOGOUT NOK enviado
isConnect ()
True
isConnect ()


2023-01-07 00:16:04,092 root            CRITICAL event:INFO,Blinks:0,COUNTER:0,EAR:0.3164692083413343
2023-01-07 00:16:04,093 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,mar:0.0
2023-01-07 00:16:04,129 root            CRITICAL event:INFO,Blinks:0,COUNTER:0,EAR:0.35135582885883615
2023-01-07 00:16:04,130 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,mar:0.03556472460805307
2023-01-07 00:16:04,180 root            CRITICAL event:INFO,Blinks:0,COUNTER:0,EAR:0.3167949609184859
2023-01-07 00:16:04,181 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,mar:0.03555700908033281
2023-01-07 00:16:04,225 root            CRITICAL event:INFO,Blinks:0,COUNTER:0,EAR:0.26535446295270193
2023-01-07 00:16:04,225 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,mar:0.010416666666666666
2023-01-07 00:16:04,270 root            CRITICAL event:INFO,Blinks:0,COUNTER:0,EAR:0.2552333500546107
2023-01-07 00:16:04,271 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,ma

Mensaje LOGIN OK enviado
CONNECT_HIVEMQTT: True
